# ANN for Regression - MLPRegressor with Hyperparameter Optimization

In [1]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from sklearn.preprocessing import label_binarize
from scipy.special import softmax

train_data = pd.read_csv("Train_Set.csv")
val_data = pd.read_csv("Validation_Set.csv")
test_data = pd.read_csv('Test_Set.csv')

X = train_data.drop(columns=["Severity"])
y = train_data["Severity"]

X_test = test_data.drop(columns=["Severity"])
y_test = test_data["Severity"]

combined_data = pd.concat([train_data, val_data], axis=0)
X_combined = combined_data.drop(columns=["Severity"])
y_combined = combined_data["Severity"]

# Create an indicator array for the validation set split
split_index = [-1] * len(train_data) + [0] * len(val_data)
predefined_split = PredefinedSplit(test_fold=split_index)

# Define the parameter grid for MLPRegressor
param_grid_mlp = {
    'hidden_layer_sizes': [(100,)],
    'alpha': [0.0001],
    'learning_rate': ['constant']
}

# Perform Grid Search for Hyperparameter Optimization
grid_search_mlp = RandomizedSearchCV(
    MLPRegressor(max_iter=200, random_state=42),
    param_grid_mlp,
    n_iter=1,
    cv=predefined_split,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search_mlp.fit(X_combined, y_combined)  

# Get the best model and parameters
best_mlp_model = grid_search_mlp.best_estimator_
print("Best parameters for MLP:", grid_search_mlp.best_params_)

# Predictions using the optimized MLPRegressor
y_pred_mlp = best_mlp_model.predict(X_test)
y_pred_mlp_rounded = y_pred_mlp.round().astype(int)

# Map predictions to the nearest valid class
valid_classes = np.array([1, 2, 3, 4])

# Function to find the nearest class
def map_to_nearest_class(predictions, valid_classes):
    return valid_classes[np.argmin(np.abs(predictions[:, np.newaxis] - valid_classes), axis=1)]

# Adjust predictions to the nearest valid class
y_pred_mlp_mapped = map_to_nearest_class(y_pred_mlp, valid_classes)

# Binarize the true labels for ROC curve (with valid classes only)
y_test_binarized = label_binarize(y_test, classes=valid_classes)
n_classes = y_test_binarized.shape[1]

# Evaluation with mapped predictions
mae_mlp = mean_absolute_error(y_test, y_pred_mlp_mapped)
mse_mlp = mean_squared_error(y_test, y_pred_mlp_mapped)
accuracy = accuracy_score(y_test, y_pred_mlp_mapped)
r2 = r2_score(y_test, y_pred_mlp_mapped)
classification_report_str = classification_report(y_test, y_pred_mlp_mapped, labels=valid_classes)

# Output the corrected results
print(f"MLP Regression - MAE: {mae_mlp}")
print(f"MLP Regression - MSE: {mse_mlp}")
print(f"Accuracy: {accuracy:.4f}")
print(f"R-squared (R²): {r2:.4f}")
print("Classification Report:")
print(classification_report_str)

Best parameters for MLP: {'learning_rate': 'constant', 'hidden_layer_sizes': (100,), 'alpha': 0.0001}
MLP Regression - MAE: 0.558781513102367
MLP Regression - MSE: 0.5984186657173319
Accuracy: 0.4610
R-squared (R²): -1.5587
Classification Report:
              precision    recall  f1-score   support

           1       0.14      0.21      0.17      9362
           2       0.82      0.45      0.58    873568
           3       0.18      0.57      0.28    188164
           4       0.17      0.14      0.16     27219

    accuracy                           0.46   1098313
   macro avg       0.33      0.34      0.30   1098313
weighted avg       0.69      0.46      0.52   1098313

